In [2]:
import pandas as pd
from datetime import timedelta
from collections import Counter
from window_generator import generate_windows_pd as wg

cols = ['date','ref_hash', 'event_id','ref_type','application_id','attributed','device_os','device_os_version','device_brand',
       'device_model', 'device_city', 'carrier', 'wifi', 'device_language']
events = pd.read_csv('./data/events.csv', usecols = cols)

events['date'] = pd.to_datetime(events['date'])
events['number_of_events'] = 1

events['attributed'] = events['attributed'].apply(lambda x: 1 if (x==True) else 0 )
events['wifi'] = events['wifi'].apply(lambda x: 1 if (x==True) else 0 )


events.head()

,date,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,device_city,carrier,device_os,wifi,device_language,number_of_events
0,2019-04-20 01:42:49.120,0,1891515180541284343,5857744372586891366,210,0,NaN,NaN,4.318294e+18,NaN,NaN,NaN,0,3.301378e+18,1
1,2019-04-20 01:42:49.340,1,1891515180541284343,7642521036780133571,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
2,2019-04-20 01:42:49.365,1,1891515180541284343,2548841562898283198,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
3,2019-04-20 01:42:51.438,2,1891515180541284343,609402887625919085,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
4,2019-04-20 01:42:51.838,1,1891515180541284343,9114651763556439823,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1


In [3]:
ew1, ew2, ew3, ew4, ew5, ew6, ew7 = wg(events, 'date')
ew1.head()

/home/fedoremelianenko/FIUBA/Organizacion de datos/TPs/TP2/window_generator.py:18: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  window1 = window1.loc[df[str_timestamp] > INITIAL_DATE]


,date,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,device_city,carrier,device_os,wifi,device_language,number_of_events
0,2019-04-20 01:42:49.120,0,1891515180541284343,5857744372586891366,210,0,NaN,NaN,4.318294e+18,NaN,NaN,NaN,0,3.301378e+18,1
1,2019-04-20 01:42:49.340,1,1891515180541284343,7642521036780133571,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
2,2019-04-20 01:42:49.365,1,1891515180541284343,2548841562898283198,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
3,2019-04-20 01:42:51.438,2,1891515180541284343,609402887625919085,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1
4,2019-04-20 01:42:51.838,1,1891515180541284343,9114651763556439823,210,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1


In [5]:
window1 = ew1.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window1.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'})

,number_of_events,device_model,carrier,device_os_version,event_id,device_os,device_brand,ref_type,device_city,attributed,wifi,device_language,application_id
ref_hash,,,,,,,,,,,,,
41863526108385,88,1.658417e+18,NaN,NaN,1,NaN,6.115026e+18,1891515180541284343,NaN,0.0,0.0,6.977049e+18,210
161514654074162,8,6.684966e+18,NaN,NaN,23,NaN,2.208835e+18,1891515180541284343,NaN,0.0,1.0,6.977049e+18,121
186034136943920,13,NaN,NaN,NaN,1,NaN,NaN,1891515180541284343,NaN,0.0,0.0,NaN,210
360710529886978,3,5.660121e+18,8.707893e+18,7.391844e+18,23,NaN,6.115026e+18,1891515180541284343,4.577311e+18,0.0,0.0,3.301378e+18,14
365882020742330,57,1.805456e+18,NaN,NaN,15,NaN,3.083059e+17,1891515180541284343,NaN,0.0,0.0,6.977049e+18,302


In [7]:
def get_mode(values):
    c = Counter(values)
    return c.most_common(1)[0][0]

window2 = ew2.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window2.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

window3 = ew3.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window3.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

window4 = ew4.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window4.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

window5 = ew5.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window5.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

window6 = ew6.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window6.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

window7 = ew7.groupby('ref_hash').agg({'event_id':get_mode,'ref_type':get_mode,'application_id':get_mode,
                                       'attributed':'mean','device_os_version':get_mode,'device_brand':get_mode,
                                      'device_model':get_mode,'device_city':get_mode,'carrier':get_mode,
                                      'device_os':get_mode,'wifi':'mean','device_language':get_mode,
                                       'number_of_events': sum})
window7.rename(columns={'device_model':'most_used_model','carrier':'most_used_carrier',
                        'device_os_version':'os_version_mode','event_id':'event_id_mode',
                       'device_os':'os_mode','ref_type':'id_type_mode','device_city':'city_mode'},inplace=True)

In [8]:
window1.to_csv('./data/features/events/ev1.csv')
window2.to_csv('./data/features/events/ev2.csv')
window3.to_csv('./data/features/events/ev3.csv')
window4.to_csv('./data/features/events/ev4.csv')
window5.to_csv('./data/features/events/ev5.csv')
window6.to_csv('./data/features/events/ev6.csv')
window7.to_csv('./data/features/events/ev7.csv')